# Teacher Model Training

Code authored by: Shaw Talebi

[Video](https://youtu.be/FLkUOkeMd5M) <br>
[Blog](https://towardsdatascience.com/compressing-large-language-models-llms-9f406eea5b5e) <br>
Based on example [here](https://huggingface.co/docs/transformers/en/tasks/sequence_classification)

### imports

In [1]:
import pandas as pd
from datasets import DatasetDict, Dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### load data

In [2]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")

In [5]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

### Train Teacher Model

In [6]:
# Load model directly
model_path = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_path, 
                                                           num_labels=2, 
                                                           id2label=id2label, 
                                                           label2id=label2id,)

/Users/shawhin/opt/anaconda3/envs/model-compression-env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Freeze base model

In [7]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight True
bert.embeddings.position_embeddings.weight True
bert.embeddings.token_type_embeddings.weight True
bert.embeddings.LayerNorm.weight True
bert.embeddings.LayerNorm.bias True
bert.encoder.layer.0.attention.self.query.weight True
bert.encoder.layer.0.attention.self.query.bias True
bert.encoder.layer.0.attention.self.key.weight True
bert.encoder.layer.0.attention.self.key.bias True
bert.encoder.layer.0.attention.self.value.weight True
bert.encoder.layer.0.attention.self.value.bias True
bert.encoder.layer.0.attention.output.dense.weight True
bert.encoder.layer.0.attention.output.dense.bias True
bert.encoder.layer.0.attention.output.LayerNorm.weight True
bert.encoder.layer.0.attention.output.LayerNorm.bias True
bert.encoder.layer.0.intermediate.dense.weight True
bert.encoder.layer.0.intermediate.dense.bias True
bert.encoder.layer.0.output.dense.weight True
bert.encoder.layer.0.output.dense.bias True
bert.encoder.layer.0.output.LayerNorm.weight True


In [8]:
# freeze base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [9]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.output.dense.bias False
bert.encoder.layer.0.output.Lay

#### Preprocess text

In [10]:
# define text preprocessing
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [11]:
# tokenize all datasetse
tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [12]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#### Evaluation

In [13]:
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred
    
    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)
    
    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)
    
    return {"Accuracy": acc, "AUC": auc}

#### Train model

In [14]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/2630 [00:00<?, ?it/s]

{'loss': 0.4916, 'grad_norm': 1.5799760818481445, 'learning_rate': 0.00018, 'epoch': 1.0}


  0%|          | 0/57 [00:00<?, ?it/s]

{'eval_loss': 0.42275097966194153, 'eval_Accuracy': 0.784, 'eval_AUC': 0.915, 'eval_runtime': 14.4369, 'eval_samples_per_second': 31.17, 'eval_steps_per_second': 3.948, 'epoch': 1.0}
{'loss': 0.3894, 'grad_norm': 2.128021478652954, 'learning_rate': 0.00016, 'epoch': 2.0}


  0%|          | 0/57 [00:00<?, ?it/s]

...

  0%|          | 0/57 [00:00<?, ?it/s]

{'eval_loss': 0.2881464660167694, 'eval_Accuracy': 0.867, 'eval_AUC': 0.951, 'eval_runtime': 6.103, 'eval_samples_per_second': 73.734, 'eval_steps_per_second': 9.34, 'epoch': 9.0}
{'loss': 0.3061, 'grad_norm': 4.887284278869629, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/57 [00:00<?, ?it/s]

{'eval_loss': 0.2963337004184723, 'eval_Accuracy': 0.873, 'eval_AUC': 0.951, 'eval_runtime': 6.2822, 'eval_samples_per_second': 71.631, 'eval_steps_per_second': 9.073, 'epoch': 10.0}
{'train_runtime': 418.2572, 'train_samples_per_second': 50.208, 'train_steps_per_second': 6.288, 'train_loss': 0.35921568199708886, 'epoch': 10.0}


TrainOutput(global_step=2630, training_loss=0.35921568199708886, metrics={'train_runtime': 418.2572, 'train_samples_per_second': 50.208, 'train_steps_per_second': 6.288, 'total_flos': 706603239165360.0, 'train_loss': 0.35921568199708886, 'epoch': 10.0})

### Apply Model to Validation Dataset

In [28]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)

  0%|          | 0/57 [00:00<?, ?it/s]

{'Accuracy': 0.889, 'AUC': 0.946}


### Push to hub

In [34]:
# push model to hub
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shawhin/bert-phishing-classifier_teacher/commit/6e4110db0febcd143d945e86d8e0ec8a08204d4c', commit_message='End of training', commit_description='', oid='6e4110db0febcd143d945e86d8e0ec8a08204d4c', pr_url=None, pr_revision=None, pr_num=None)